# Introduction

This tutorial introduces the basic Auto-PyTorch API together with the classes for featurized and image data.
So far, Auto-PyTorch covers classification and regression on featurized data as well as classification on image data.
For installing Auto-PyTorch, please refer to the github page.

**Disclaimer**: In this notebook, data will be downloaded from the openml project for featurized tasks and CIFAR10 will be downloaded for image classification. Hence, an internet connection is required.

# API

There are classes for featurized tasks (classification, multi-label classification, regression) and image tasks (classification). You can import them via:

In [1]:
import sys

path = ['/home/mikel/Desktop/Work/Auto-PyTorch', '/home/mikel/Desktop/Work/Auto-PyTorch/examples/basics', '/home/mikel/bin/caffe/cvpr16-deepbit-master/python', '/home/mikel/miniconda3/lib/python37.zip', '/home/mikel/miniconda3/lib/python3.7', '/home/mikel/miniconda3/lib/python3.7/lib-dynload', '', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/IPython/extensions', '/home/mikel/.ipython']
for ele in path:
    sys.path.insert(0, ele)

import os
sys.path.insert(0,"/home/mikel/Desktop/Work/Auto-PyTorch")
print(os.getcwd())

print(sys.path)

/home/mikel/Desktop/Work/Auto-PyTorch/examples/basics
['/home/mikel/Desktop/Work/Auto-PyTorch', '/home/mikel/.ipython', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/IPython/extensions', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg', '/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages', '', '/home/mikel/miniconda3/lib/python3.7/lib-dynload', '/home/mikel/miniconda3/lib/python3.7', '/home/mikel/miniconda3/lib/python37.zip', '/home/mikel/bin/caffe/cvpr16-deepbit-master/python', '/home/mikel/Desktop/Work/Auto-PyTorch/examples/basics', '/home/mikel/Desktop/Work/Auto-PyTorch', '/home/mikel/Desktop/Work/Auto-PyTorch/examples/basics', '/tmp/846fd34b-33e9-4e2b-94ee-cf32113786fa', '/home/mikel/bin/caffe/cvpr16-deepbit-master/python', '/home/mikel/.vscode/extensions/ms-python.python-2020.6.90262/pythonFiles', '/home/mikel/.vscode/extensions/ms-python.python-2020.6.90262/pythonFiles/lib/python', '/ho

In [2]:
from autoPyTorch import (AutoNetClassification,
                         AutoNetMultilabel,
                         AutoNetRegression,
                         AutoNetImageClassification,
                         AutoNetImageClassificationMultipleDatasets)

Editing!


In [3]:
# Other imports for later usage
import pandas as pd
import numpy as np
import os as os
import openml
import json

Upon initialization of a class, you can specify its configuration. Later, you can override its configuration in each fit call. The *config_preset* allows to constrain the search space to one of *tiny_cs, medium_cs* or *full_cs*. These presets can be seen in *core/presets/*.

In [4]:
autonet = AutoNetClassification(config_preset="tiny_cs", result_logger_dir="logs/")

Here are some useful methods provided by the API:

In [5]:
# Get the current configuration as dict
current_configuration = autonet.get_current_autonet_config()

# Get the ConfigSpace object with all hyperparameters, conditions, default values and default ranges
hyperparameter_search_space = autonet.get_hyperparameter_search_space()

# Print all possible configuration options 
#autonet.print_help()

The most important methods for using Auto-PyTorch are ***fit***, ***refit***, ***score*** and ***predict***.

First, we get some data:

In [6]:
# Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

***fit*** is used to search for a good configuration by fitting configurations chosen by the algorithm (by default BOHB). The incumbent configuration is then returned and stored in the class.

We recommend to have a look at the possible configuration options first. Some of the most important options allow you to set the budget type (epochs or time), run id and task id for cluster usage, tensorboard logging, seed and more.

Here we search for a configuration for 300 seconds with 60-100 s time for fitting each individual configuration.
Use the *validation_split* parameter to specify a split size. You can also pass your own validation set
via *X_val* and *Y_val*. Use *log_level="info"* or *log_level="debug"* for more detailed output.

In [7]:
autonet = AutoNetClassification(config_preset="tiny_cs", result_logger_dir="logs/", use_tensorboard_logger=True)
# Fit (note that the settings are for demonstration, you might need larger budgets)
results_fit = autonet.fit(X_train=X_train,
                          Y_train=Y_train,
                          validation_split=0.3,
                          max_runtime=300,
                          min_budget=60,
                          max_budget=100,
                          refit=True)

# Save fit results as json
with open("logs/results_fit.json", "w") as file:
    json.dump(results_fit, file)

data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
data saved in: logs/Train/
d

KeyboardInterrupt: 

***refit*** allows you to fit a configuration of your choice for a defined time. By default, the incumbent configuration is refitted during a *fit* call using the *max_budget*. However, *refit* might be useful if you want to fit on the full dataset or even another dataset or if you just want to fit a model without searching.

You can specify a hyperparameter configuration to fit (if you do not specify a configuration the incumbent configuration from the last fit call will be used):

In [8]:
# Create an autonet
autonet_config = {
    "result_logger_dir" : "logs/",
    "budget_type" : "epochs",
    "log_level" : "info", 
    "use_tensorboard_logger" : True,
    "validation_split" : 0.0
    }
autonet = AutoNetClassification(**autonet_config)

# Sample a random hyperparameter configuration as an example
hyperparameter_config = autonet.get_hyperparameter_search_space().sample_configuration().get_dictionary()

# Refit with sampled hyperparameter config for 120 s. This time on the full dataset.
results_refit = autonet.refit(X_train=X_train,
                              Y_train=Y_train,
                              X_valid=None,
                              Y_valid=None,
                              hyperparameter_config=hyperparameter_config,
                              autonet_config=autonet.get_current_autonet_config(),
                              budget=50)

# Save json
with open("logs/results_refit.json", "w") as file:
    json.dump(results_refit, file)

09:14:53 Start autonet with config:
{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing', 'plateau'], 'networks': ['shapedresnet'], 'over_sampling_methods': ['smote'], 'preprocessors': ['none', 'truncated_svd', 'power_transformer'], 'target_size_strategies': ['none', 'upsample', 'median'], 'result_logger_dir': 'logs/', 'budget_type': 'epochs', 'log_level': 'info', 'use_tensorboard_logger': True, 'validation_split': 0.0, 'hyperparameter_search_space_updates': None, 'categorical_features': None, 'dataset_name': None, 'run_id': '0', 'task_id': -1, 'algorithm': 'bohb', 'portfolio_type': 'greedy', 'eta': 3, 'min_workers': 1, 'working_dir': '.', 'network_interface_name': 'wlp3s0', 'memory_limit_mb': 1000000, 'run_worker_on_master_node': True, 'use_pynisher': True, 'refit_validation_split': 0.0, 'cross_validator': 'none', 'cross_validator_args': {}, 'min_budget_for_cv': 0, 'shuffle': True, 'imputation_strategies': ['mean', 'median', 'most_frequent'], 'normalization_strategies': ['none

***pred*** returns the predictions of the incumbent model. ***score*** can be used to evaluate the model on a test set. 

In [9]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

Model prediction: [[0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]]
Accuracy score 70.0


Finally, you can also get the incumbent model as PyTorch Sequential model via

In [12]:
pytorch_model = autonet.get_pytorch_model()
print(pytorch_model)

Sequential(
  (0): Linear(in_features=19, out_features=187, bias=True)
  (1): Sequential(
    (0): ResBlock(
      (shortcut): Linear(in_features=187, out_features=135, bias=True)
      (start_norm): Sequential(
        (0): BatchNorm1d(187, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Tanh()
      )
      (layers): Sequential(
        (0): Linear(in_features=187, out_features=135, bias=True)
        (1): BatchNorm1d(135, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Tanh()
        (3): Dropout(p=True, inplace=False)
        (4): Linear(in_features=135, out_features=135, bias=True)
      )
      (shake_shake_layers): Sequential(
        (0): Linear(in_features=187, out_features=135, bias=True)
        (1): BatchNorm1d(135, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Tanh()
        (3): Dropout(p=True, inplace=False)
        (4): Linear(in_features=135, out_features=135, bias=True)
      )
  

# Featurized Data

All classes for featurized data (*AutoNetClassification*, *AutoNetMultilabel*, *AutoNetRegression*) can be used as in the example above. The only difference is the type of labels they accept.

# Image Data

Auto-PyTorch provides two classes for image data. *autonet_image_classification* can be used for classification for images. The *autonet_multi_image_classification* class allows to search for configurations for image classification across multiple datasets. This means Auto-PyTorch will try to choose a configuration that works well on all given datasets.

In [9]:
# Load classes
autonet_image_classification = AutoNetImageClassification(config_preset="full_cs", result_logger_dir="logs/",use_tensorboard_logger=True, tensorboard_min_log_interval=1)
#autonet_multi_image_classification = AutoNetImageClassificationMultipleDatasets(config_preset="tiny_cs", result_logger_dir="logs/")

For passing your image data, you have two options (note that arrays are expected):

I) Via a path to a comma-separated value file, which in turn contains the paths to the images and the image labels (note header is assumed to be None):

In [10]:
csv_dir = os.path.abspath("../../datasets/example.csv")

X_train = np.array([csv_dir])
Y_train = np.array([0])

II) directly passing the paths to the images and the labels

In [11]:
df = pd.read_csv(csv_dir, header=None)
X_train = df.values[:,0]
Y_train = df.values[:,1]

Make sure you specify *image_root_folders* if the paths to the images are not specified from your current working directory. You can also specify *images_shape* to up- or downscale images.

Using the flag *save_checkpoints=True* will save checkpoints to the result directory:

In [12]:
autonet_image_classification.fit(X_train=X_train,
                                 Y_train=Y_train,
                                 images_shape=[3,32,32],
                                 min_budget=200,
                                 max_budget=600,
                                 max_runtime=10000,
                                 save_checkpoints=True,
                                 images_root_folders=[os.path.abspath("../../datasets/example_images")])

m.py", line 135, in setstate
    version = state[0]
TypeError: 'float' object is not subscriptable
11:54:06 job (300, 1, 0) failed with exception
Traceback (most recent call last):
  File "/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/mikel/Desktop/Work/Auto-PyTorch/autoPyTorch/core/worker_no_timelimit.py", line 71, in compute
    raise Exception("Exception in train pipeline. Took " + str((time.time()-start_time)) + " seconds with budget " + str(budget))
Exception: Exception in train pipeline. Took 0.05415678024291992 seconds with budget 600.0

Process pynisher function call:
Traceback (most recent call last):
  File "/home/mikel/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/mikel/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._tar

RuntimeError: No models fit during training, please retry with a larger max_runtime.

Auto-PyTorch also supports some common datasets. By passing a comma-separated value file with just one line, e.g. "CIFAR10, 0" and specifying *default_dataset_download_dir* it will automatically download the data and use it for searching. Supported datasets are CIFAR10, CIFAR100, SVHN and MNIST.

In [9]:
path_to_cifar_csv = os.path.abspath("../../datasets/CIFAR10.csv")

autonet_image_classification.fit(X_train=np.array([path_to_cifar_csv]),
                                 Y_train=np.array([0]),
                                 min_budget=600,
                                 max_budget=900,
                                 max_runtime=600,
                                 default_dataset_download_dir="./datasets",
                                 images_root_folders=["./datasets"])

rain pipeline. Took 0.06778836250305176 seconds with budget 900.0

Process pynisher function call:
Traceback (most recent call last):
  File "/home/mikel/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/mikel/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mikel/Desktop/Work/Auto-PyTorch/env/lib/python3.7/site-packages/pynisher/limit_function_call.py", line 93, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/home/mikel/Desktop/Work/Auto-PyTorch/autoPyTorch/core/worker_no_timelimit.py", line 108, in optimize_pipeline
    random.setstate(random_state)
  File "/home/mikel/miniconda3/lib/python3.7/random.py", line 135, in setstate
    version = state[0]
TypeError: 'float' object is not subscriptable
10:39:47 job (220, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/mikel/Desktop/Work/Auto-PyTorch

KeyboardInterrupt: 

For searching across multiple datasets, pass multiple csv files to the corresponding Auto-PyTorch class. Make sure your specify *images_root_folders* for each of them.

In [ ]:
autonet_multi_image_classification.fit(X_train=np.array([path_to_cifar_csv, csv_dir]),
                                       Y_train=np.array([0]),
                                       min_budget=1500,
                                       max_budget=2000,
                                       max_runtime=4000,
                                       default_dataset_download_dir="./datasets",
                                       images_root_folders=["./datasets", "./datasets/example_images"])